In [ ]:
import glob
import re

from lxml import etree as ET

from teipy import TeiReader

## add @ref to tei:author
* go through all TEIs
* in each author node, find the email node, slugify the value
* use this for the tei:author's @ref

In [ ]:
files = glob.glob("../dhd_*/TEI/*.xml")
# len(files)

In [ ]:
for x in files:
    doc = TeiReader(x)
    [x for x in doc.add_refs()]
    doc.tree_to_file(file=x)

## add the year of the conference somewhere

* the year/conference of the abstract is not encoded in a homogenous manner - sigh - so this needs to be added as well

In [ ]:
for x in files:
    doc = TeiReader(x)
    # delete any already existing title @type=conf_year to avoid adding a new title element each time this script is run
    try:
        conf_el_remove = doc.tree.xpath(".//tei:title[@type='conf_year']", namespaces=doc.ns_tei)[0]
#         print('delete already existing conf_el element')
    except IndexError:
#         print('no conf_el created yet')
        conf_el_remove = False
    if conf_el_remove:
        conf_el_remove.getparent().remove(conf_el_remove) 
    xpath_expr = ".//tei:titleStmt//tei:title"
    last_title = doc.tree.xpath(xpath_expr, namespaces=doc.ns_tei)[-1]
    conf_year = re.findall('dhd_\d\d\d\d', x)[0][4:]
    conf_el = ET.Element("{http://www.tei-c.org/ns/1.0}title")
    conf_el.attrib['type'] = 'conf_year'
    conf_el.text = f"{conf_year}"
    last_title.addnext(conf_el)
    title_stmt = doc.tree.xpath(".//tei:titleStmt", namespaces=doc.ns_tei)[0]
    doc.tree_to_file(file=x)